# NHL DATA SCRAPER v.2.0

### Step 1
Import all required modules

In [1]:
import os
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Step 2
1. Global variables
2. Set up chrome options for automatic download

In [2]:
DOWNLOADS_PATH = 'C:\\Users\\riger\\Downloads'
SCRIPT_PATH = os.getcwd()
NHL_STATS_CSV_PATH = os.path.join(SCRIPT_PATH, './generated/nhl_stats_data.csv')
NHL_GOALIE_CSV_PATH = os.path.join(SCRIPT_PATH, './generated/nhl_goalie_data.csv')
COMBINED_CSV_PATH = os.path.join(SCRIPT_PATH, './generated/final_nhl_stats.csv')
BASE_STATS_URL = 'https://www.nhl.com/stats/teams?aggregate=0&report=daysbetweengames&reportType=game&seasonFrom=20222023&seasonTo=20242025&dateFromSeason&gameType=2&sort=a_gameDate&page={}&pageSize=100'
BASE_GOALIE_URL = 'https://www.nhl.com/stats/goalies?aggregate=0&reportType=game&seasonFrom=20222023&seasonTo=20242025&dateFromSeason&gameType=2&sort=a_gameDate&page={}&pageSize=100'

In [3]:
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': DOWNLOADS_PATH,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

### Step 4
Start scraping stats
- This will open every page from 0 to 52
- It will download the csv to your downloads folder, read it, and save/concat it to the 'data' pandas df
- Finally, remove the downloaded file
- Repeat with all pages

In [8]:
#Driver setup

service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

In [9]:
try:
    # Create csv directory if doesn't exist (will be used at the end)
    if not os.path.exists(NHL_STATS_CSV_PATH):
        os.makedirs('generated', exist_ok=True)  # create 'generated' folder if needed
        
        with open(NHL_STATS_CSV_PATH, 'w') as f:
            f.write("")
            print("Created file:", NHL_STATS_CSV_PATH)

    combined_data = pd.DataFrame()

    for page in range(0, 79):
        url = BASE_STATS_URL.format(page)
        driver.get(url)

        # Wait for the export link to be present and clickable
        export_link = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
        )

        # Scroll to the element
        driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
        export_link.click()
        time.sleep(3)  # Wait for the file to download

        # Process the downloaded file
        downloaded_file = os.path.join(DOWNLOADS_PATH, 'Days between Games.xlsx')
        if os.path.exists(downloaded_file):
            data = pd.read_excel(downloaded_file)
            combined_data = pd.concat([combined_data, data], ignore_index=True)
            os.remove(downloaded_file)  # Remove the downloaded file after processing

    # Save the combined data as a CSV file
    if os.path.exists('Days between Games.xlsx'):
        os.remove('Days between Games.xlsx')

    combined_data.to_csv(NHL_STATS_CSV_PATH, index=False)
    print(f"Combined CSV saved at: {NHL_STATS_CSV_PATH}")

finally:
    driver.quit()

Combined CSV saved at: c:\Users\riger\Desktop\projects\NHL_predictor\main\ml_dev\notebooks\./generated/nhl_stats_data.csv


### Step 5
Scrape goalie data
* This will scrape all pages from 0 to 56
* Like before, will download a csv, read into a pandas df
* combine all pages into one csv

In [10]:
try:

    # Start the Chrome driver
    service = Service('chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    # Create csv directory if doesn't exist (will be used at the end)
    if not os.path.exists(NHL_GOALIE_CSV_PATH):
        if not os.path.exists('generated'):
            os.makedirs('generated', exist_ok=True)  # create 'generated' folder if needed
        
        with open(NHL_GOALIE_CSV_PATH, 'w') as f:
            f.write("")
            print("Created file:", NHL_GOALIE_CSV_PATH)

    combined_data = pd.DataFrame()

    for page in range(0, 84):
        url = BASE_GOALIE_URL.format(page)
        driver.get(url)

        # Wait for the export link to be present and clickable
        export_link = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/main/div/div/div/div[2]/div/div[2]/div/main/div[2]/h4/a'))
        )

        # Scroll to the element
        driver.execute_script("arguments[0].scrollIntoView(true);", export_link)
        export_link.click()
        time.sleep(4)  # Wait for the file to download

        # Process the downloaded file
        downloaded_file = os.path.join(DOWNLOADS_PATH, 'Summary.xlsx')
        if os.path.exists(downloaded_file):
            data = pd.read_excel(downloaded_file)
            combined_data = pd.concat([combined_data, data], ignore_index=True)
            os.remove(downloaded_file)  # Remove the downloaded file after processing

    # Save the combined data as a CSV file
    if os.path.exists('Summary.xlsx'):
        os.remove('Summary.xlsx')

    combined_data.to_csv(NHL_GOALIE_CSV_PATH, index=False)
    print(f"Combined CSV saved at: {NHL_GOALIE_CSV_PATH}")

finally:
    driver.quit()

Combined CSV saved at: c:\Users\riger\Desktop\projects\NHL_predictor\main\ml_dev\notebooks\./generated/nhl_goalie_data.csv


### Step 6
Filter goalie data
* loop through every combination of date + team
* If result has 2 or more rows, it means there was a goalie switch. reduce to one row
    * to do this, apply this example formula to calculate save %:
        * team_save_percentage = (saves_ullmark + saves_forsberg) / (shots_against_ullmark + shots_against_forsberg)
    * to decide on the name that stays, compare ice times
* **In the end, number of rows in goalie data should match number of rows in stats data**

In [11]:
goalie_df = pd.read_csv('./generated/nhl_goalie_data.csv')

In [12]:
def toi_to_seconds(toi_str):
    mins, secs = map(int, toi_str.split(':'))
    return mins * 60 + secs

goalie_df['TOI_seconds'] = goalie_df['TOI'].apply(toi_to_seconds)

In [13]:
grouped = goalie_df.groupby(['Game Date', 'Team'])

# Iterate through each group
for (game_date, team), group in grouped:
    if group.shape[0] > 1:
        curr_saves = group['Svs'].sum()
        curr_shots = group['SA'].sum()

        # Get goalie with most TOI
        max_toi_row = group.loc[group['TOI_seconds'].idxmax()]
        goalie = max_toi_row['Player']

        # Drop other rows
        goalie_df = goalie_df.drop(group.index[group['Player'] != goalie])

        # Update save percentage
        new_sv_percentage = round(curr_saves / curr_shots, 3)
        goalie_df.loc[group.index[group['Player'] == goalie], 'Sv%'] = new_sv_percentage

# Save the modified DataFrame to a new CSV file
goalie_df.to_csv(NHL_GOALIE_CSV_PATH, index=False)

#### Step 6.1
Expand team name in goalie data to be easily combined with team name (home) in stat data

In [14]:
nhl_team_abbr = {
    'ANA' : 'Anaheim Ducks',
    'ARI' : 'Utah Hockey Club',
    'BOS' : 'Boston Bruins',
    'BUF' : 'Buffalo Sabres',
    'CAR' : 'Carolina Hurricanes',
    'CBJ' : 'Columbus Blue Jackets',
    'CGY' : 'Calgary Flames',
    'CHI' : 'Chicago Blackhawks',
    'COL' : 'Colorado Avalanche',
    'DAL' : 'Dallas Stars',
    'DET' : 'Detroit Red Wings',
    'EDM' : 'Edmonton Oilers',
    'FLA' : 'Florida Panthers',
    'LAK' : 'Los Angeles Kings',
    'MIN' : 'Minnesota Wild',
    'MTL' : 'Montréal Canadiens',
    'NJD' : 'New Jersey Devils',
    'NSH' : 'Nashville Predators',
    'NYI' : 'New York Islanders',
    'NYR' : 'New York Rangers',
    'OTT' : 'Ottawa Senators',
    'PHI' : 'Philadelphia Flyers',
    'PIT' : 'Pittsburgh Penguins',
    'SJS' : 'San Jose Sharks',
    'SEA' : 'Seattle Kraken',
    'STL' : 'St. Louis Blues',
    'TBL' : 'Tampa Bay Lightning',
    'TOR' : 'Toronto Maple Leafs',
    'UTA' : 'Utah Hockey Club',
    'VAN' : 'Vancouver Canucks',
    'VGK' : 'Vegas Golden Knights',
    'WPG' : 'Winnipeg Jets',
    'WSH' : 'Washington Capitals',
}

accidental_inconcictent_naming = {
    'Arizona Coyotes' : 'Utah Hockey Club',
    'ARI' : 'UTA',
}

In [15]:
stats_df = pd.read_csv('./generated/nhl_stats_data.csv')
goalie_df = pd.read_csv('./generated/nhl_goalie_data.csv')

In [16]:
stats_df['Team'] = stats_df['Team'].apply(lambda abbr: accidental_inconcictent_naming.get(abbr, abbr))
stats_df['Opp Team'] = stats_df['Opp Team'].apply(lambda abbr: nhl_team_abbr.get(abbr, abbr))
stats_df.to_csv(NHL_STATS_CSV_PATH, index=False)

goalie_df['Team'] = goalie_df['Team'].apply(lambda abbr: nhl_team_abbr.get(abbr, abbr))
goalie_df.to_csv(NHL_GOALIE_CSV_PATH, index=False)

### Step 7
Combine **goalie data (name, sv%)** with **stats data** based on data and home team name
* Create a final csv file called 'final_nhl_stats.csv'

In [17]:
stats_df = pd.read_csv('./generated/nhl_stats_data.csv')
goalie_df = pd.read_csv('./generated/nhl_goalie_data.csv')

In [18]:
goalie_subset = goalie_df[['Game Date', 'Team', 'Player', 'Sv%']]

final_nhl_stats_df = pd.merge(stats_df, goalie_subset, on=['Game Date', 'Team'], how='left')

final_nhl_stats_df = final_nhl_stats_df.rename(columns={'Player': 'Goalie'})

final_nhl_stats_df.to_csv(COMBINED_CSV_PATH, index=False)

### Step 8
Mark Home and Away games

In [19]:
hockey_ref_url_2023 = 'https://www.hockey-reference.com/leagues/NHL_2023_games.html'
hockey_ref_url_2024 = 'https://www.hockey-reference.com/leagues/NHL_2024_games.html'
hockey_ref_url_2025 = 'https://www.hockey-reference.com/leagues/NHL_2025_games.html'

# Read tables for each season
hockey_ref_dfs_2023 = pd.read_html(hockey_ref_url_2023)
hockey_ref_dfs_2024 = pd.read_html(hockey_ref_url_2024)
hockey_ref_dfs_2025 = pd.read_html(hockey_ref_url_2025)

# Usually, the first table is the games table
hockey_ref_dfs_2023[0].to_csv('./generated/nhl_ref_data_2023.csv', index=False)
hockey_ref_dfs_2024[0].to_csv('./generated/nhl_ref_data_2024.csv', index=False)
hockey_ref_dfs_2025[0].to_csv('./generated/nhl_ref_data_2025.csv', index=False)

nhl_ref_data_2023 = pd.read_csv('./generated/nhl_ref_data_2023.csv')
nhl_ref_data_2024 = pd.read_csv('./generated/nhl_ref_data_2024.csv')
nhl_ref_data_2025 = pd.read_csv('./generated/nhl_ref_data_2025.csv')

In [20]:
nhl_ref_data_2023 = nhl_ref_data_2023.replace('Arizona Coyotes', 'Utah Hockey Club')
nhl_ref_data_2024 = nhl_ref_data_2024.replace('Arizona Coyotes', 'Utah Hockey Club')
nhl_ref_data_2025 = nhl_ref_data_2025.replace('Arizona Coyotes', 'Utah Hockey Club')

nhl_ref_data_2023 = nhl_ref_data_2023.replace('Montreal Canadiens', 'Montréal Canadiens')
nhl_ref_data_2024 = nhl_ref_data_2024.replace('Montreal Canadiens', 'Montréal Canadiens')
nhl_ref_data_2025 = nhl_ref_data_2025.replace('Montreal Canadiens', 'Montréal Canadiens')

In [21]:
def add_venue_column(final_nhl_stats_df, nhl_ref_data_list):
    # Combine reference data
    nhl_ref_data = pd.concat(nhl_ref_data_list, ignore_index=True)

    # Convert date columns to datetime
    nhl_ref_data['Date'] = pd.to_datetime(nhl_ref_data['Date']).dt.date
    final_nhl_stats_df['Game Date'] = pd.to_datetime(final_nhl_stats_df['Game Date']).dt.date

    # Normalize team names (optional but safe)
    nhl_ref_data['Home'] = nhl_ref_data['Home'].str.strip()
    nhl_ref_data['Visitor'] = nhl_ref_data['Visitor'].str.strip()
    final_nhl_stats_df['Team'] = final_nhl_stats_df['Team'].str.strip()

    # Build lookup
    venue_lookup = {}
    for _, row in nhl_ref_data.iterrows():
        date = row['Date']
        home = row['Home']
        visitor = row['Visitor']
        venue_lookup[(date, home)] = 'Home'
        venue_lookup[(date, visitor)] = 'Away'

    # Apply venue lookup
    def get_venue(row):
        return venue_lookup.get((row['Game Date'], row['Team']), 'Unknown')

    final_nhl_stats_df['Venue'] = final_nhl_stats_df.apply(get_venue, axis=1)
    return final_nhl_stats_df

In [22]:
nhl_ref_data_list = [nhl_ref_data_2023, nhl_ref_data_2024, nhl_ref_data_2025]
final_nhl_stats_df = add_venue_column(final_nhl_stats_df, nhl_ref_data_list)

final_nhl_stats_df.tail()

,Team,Opp Team,Game Date,Days Btwn Games,GP,W,L,T,OT,P,...,SA/GP,SD/GP,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Goalie,Sv%,Venue
7867,Washington Capitals,Pittsburgh Penguins,2025-04-17,1,1,0,1,--,0,0,...,38,-18,2,2,50,50,50.0,Clay Stevenson,0.868,Away
7868,Detroit Red Wings,Toronto Maple Leafs,2025-04-17,0,1,0,0,--,1,1,...,20,14,2,2,50,100,35.6,Cam Talbot,0.800,Away
7869,Calgary Flames,Los Angeles Kings,2025-04-17,1,1,1,0,--,0,2,...,31,-2,2,3,0,66.7,55.8,Daniel Vladar,0.968,Away
7870,Los Angeles Kings,Calgary Flames,2025-04-17,1,1,0,1,--,0,0,...,29,2,3,2,33.3,100,44.2,David Rittich,0.828,Home
7871,Columbus Blue Jackets,New York Islanders,2025-04-17,1,1,1,0,--,0,2,...,37,-11,0,1,--,100,42.9,Jet Greaves,0.973,Home


After checking, we can save to csv

In [23]:
# save to final csv
final_nhl_stats_df.to_csv(COMBINED_CSV_PATH, index=False)
print(f"Final combined CSV saved at: {COMBINED_CSV_PATH}")

Final combined CSV saved at: c:\Users\riger\Desktop\projects\NHL_predictor\main\ml_dev\notebooks\./generated/final_nhl_stats.csv


In [24]:
# delete the ref data csvs
os.remove('./generated/nhl_ref_data_2023.csv')
os.remove('./generated/nhl_ref_data_2024.csv')
os.remove('./generated/nhl_ref_data_2025.csv')

### Step 9
Combine **home** and **away** games into same row

**Strategy**
* I am choosing to identify same games by sorting the combination of 'Game Data', 'Team', and 'Opponent'
* We will make this into a temporary id columns which we can use to "join" rows together

In [25]:
final_nhl_stats_df['game_id'] = (
    pd.to_datetime(final_nhl_stats_df['Game Date']).dt.strftime('%Y-%m-%d') + '_' +
    final_nhl_stats_df[['Team', 'Opp Team']].apply(lambda x: '_'.join(sorted(x)), axis=1)
)

In [26]:
# Read the final combined CSV to verify
# final_nhl_stats_df = pd.read_csv(COMBINED_CSV_PATH)
final_nhl_stats_df.head()

,Team,Opp Team,Game Date,Days Btwn Games,GP,W,L,T,OT,P,...,SD/GP,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Goalie,Sv%,Venue,game_id
0,Nashville Predators,San Jose Sharks,2022-10-07,4,1,1,0,--,0,2,...,1,4,4,0.0,100.0,45.5,Juuse Saros,0.968,Home,2022-10-07_Nashville Predators_San Jose Sharks
1,San Jose Sharks,Nashville Predators,2022-10-07,4,1,0,1,--,0,0,...,-1,4,4,0.0,100.0,54.6,James Reimer,0.903,Away,2022-10-07_Nashville Predators_San Jose Sharks
2,Nashville Predators,San Jose Sharks,2022-10-08,0,1,1,0,--,0,2,...,-15,3,4,0.0,75.0,49.3,Kevin Lankinen,0.939,Away,2022-10-08_Nashville Predators_San Jose Sharks
3,San Jose Sharks,Nashville Predators,2022-10-08,0,1,0,1,--,0,0,...,15,4,3,25.0,100.0,50.7,Kaapo Kahkonen,0.833,Home,2022-10-08_Nashville Predators_San Jose Sharks
4,New York Rangers,Tampa Bay Lightning,2022-10-11,4,1,1,0,--,0,2,...,13,4,6,25.0,83.3,63.8,Igor Shesterkin,0.962,Home,2022-10-11_New York Rangers_Tampa Bay Lightning


Now that we confirmed the id's are working, lets group the data

In [27]:
# print out the list of columns in the final dataframe
#cols = ['Days Btwn Games', 'GF/GP', 'GA/GP', 'Shots/GP', 'SA/GP', 'PP Opp/GP', 'TS/GP', 'PP%', 'PK%', 'FOW%', 'Goalie', 'Sv%']

# group by game_id NOT BY THE COLS
grouped_df = final_nhl_stats_df.groupby('game_id').agg(lambda x: list(x)).reset_index()

grouped_df.head()

,game_id,Team,Opp Team,Game Date,Days Btwn Games,GP,W,L,T,OT,...,SA/GP,SD/GP,PP Opp/GP,TS/GP,PP%,PK%,FOW%,Goalie,Sv%,Venue
0,2022-10-07_Nashville Predators_San Jose Sharks,"[Nashville Predators, San Jose Sharks]","[San Jose Sharks, Nashville Predators]","[2022-10-07, 2022-10-07]","[4, 4]","[1, 1]","[1, 0]","[0, 1]","[--, --]","[0, 0]",...,"[31, 32]","[1, -1]","[4, 4]","[4, 4]","[0.0, 0.0]","[100.0, 100.0]","[45.5, 54.6]","[Juuse Saros, James Reimer]","[0.968, 0.903]","[Home, Away]"
1,2022-10-08_Nashville Predators_San Jose Sharks,"[Nashville Predators, San Jose Sharks]","[San Jose Sharks, Nashville Predators]","[2022-10-08, 2022-10-08]","[0, 0]","[1, 1]","[1, 0]","[0, 1]","[--, --]","[0, 0]",...,"[33, 18]","[-15, 15]","[3, 4]","[4, 3]","[0.0, 25.0]","[75.0, 100.0]","[49.3, 50.7]","[Kevin Lankinen, Kaapo Kahkonen]","[0.939, 0.833]","[Away, Home]"
2,2022-10-11_Los Angeles Kings_Vegas Golden Knights,"[Los Angeles Kings, Vegas Golden Knights]","[Vegas Golden Knights, Los Angeles Kings]","[2022-10-11, 2022-10-11]","[4, 4]","[1, 1]","[0, 1]","[1, 0]","[--, --]","[0, 0]",...,"[51, 30]","[-21, 21]","[3, 5]","[5, 3]","[0.0, 20.0]","[80.0, 100.0]","[49.2, 50.9]","[Jonathan Quick, Logan Thompson]","[0.922, 0.9]","[Home, Away]"
3,2022-10-11_New York Rangers_Tampa Bay Lightning,"[New York Rangers, Tampa Bay Lightning]","[Tampa Bay Lightning, New York Rangers]","[2022-10-11, 2022-10-11]","[4, 4]","[1, 1]","[1, 0]","[0, 1]","[--, --]","[0, 0]",...,"[26, 39]","[13, -13]","[4, 6]","[6, 4]","[25.0, 16.7]","[83.3, 75.0]","[63.8, 36.2]","[Igor Shesterkin, Andrei Vasilevskiy]","[0.962, 0.923]","[Home, Away]"
4,2022-10-12_Anaheim Ducks_Seattle Kraken,"[Anaheim Ducks, Seattle Kraken]","[Seattle Kraken, Anaheim Ducks]","[2022-10-12, 2022-10-12]","[4, 4]","[1, 1]","[1, 0]","[0, 0]","[--, --]","[0, 1]",...,"[48, 27]","[-21, 21]","[3, 5]","[5, 3]","[66.7, 60.0]","[40.0, 33.3]","[57.1, 42.9]","[John Gibson, Philipp Grubauer]","[0.917, 0.815]","[Home, Away]"
